[INDEX](Kubernetes.ipynb)


<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width="60" />
## 5. Kubernetes Services

# Exposing applications as a Service
<pre style="align:center" >
In this section we will work with a Minikube-based single-node cluster.
</pre>
<center><img src="images/KubeNodes_1mw_Minikube.svg.png" width="600" /></center>
<pre style="align:center" >
Nevertheless the following architecture diagrams will depict the more general multi-node case.
</pre>
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

## Services allow to hide Pod location
<img src="images/Service_Access.svg.png" width="800" />
<pre>
During their lifecycle the ip addresses of Pods change (due to scaling, failures ...).

Exposing our application as a Service allows to access at a known stable address.

It can also allow load-balancing across pods.
</pre>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

## Accessing application Pods via a Service

<pre>
There exist several types of Service which may be used to safely expose Pods
for external (or internal to the Cluster).

We will look at just 3 ways
- NodePort
- LoadBalancer (requires an external LoadBalancer)
- Ingress Controller
</pre>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

<a name="NodePort"> </a><h4> NodePort </h4>
<center><table align="center" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/Service_Access_NodePort.svg.png" width="600" /> </td>
    <td bgcolor="#ffffff" color="#ffffff">
        <pre style='align:normal; font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
Simple, not recommended for production<br/>
- Only one service per port
- Limited to use ports 30000 - 32767
- User must directly address Node IP</pre>
    </td></tr>
</table></center>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

<a name="LoadBalancer"> </a><h4> LoadBalancer </h4>
<center><table align="center" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/Service_Access_LoadBalancer.svg.png" width="600" /> </td>
    <td bgcolor="#ffffff" color="#ffffff">
        <pre style='align:normal; font-family:"Courier New", Courier, monospace; font-size:90%; bottom:0px;'>
Uses 'Infrastructure Provider' LoadBalancer
<br/>
OK for deployment, but
- LoadBalancer and IP per service
</pre>
    </td></tr>
</table></center>
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

<a name="IngressController"> </a><h4> Ingress Controller </h4>

<center><table align="center" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/Service_Access_Ingress.svg.png" width="600" /> </td>
    <td bgcolor="#ffffff" color="#ffffff">
        <pre style='align:normal; font-family:"Courier New", Courier, monospace; font-size:90%; bottom:0px;'>
Capabilities depend upon Ingress Controller
<br/>
- Kubernetes limited to http(s) currently
- Generally, path-based routing is allowed
to Services
</pre>
    </td></tr>
</table></center>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr>
    <td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/traefik.logo.png" width="60" /> </td>
    <td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td>
    </tr>
    </table>

<h4> Traefik - as an Ingress Controller </h4>

<table align="left" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/Service_Access_Ingress.svg.png" width="300" /> </td>
    <td bgcolor="#ffffff" color="#ffffff">
        <p style='align:left; font-family:"Courier New", Courier, monospace; font-size:90%; bottom:0px;'>
Let's look at the use of Traefik which allows
<br/>
- Extensive Host, Path, Port-based routing to Services<br/>
- Hot configuration reloads<br/>
- Let's Encrypt Support (Automated HTTPS with renewal)<br/>
</p>
    </td></tr>
</table>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr>
    <td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/traefik.logo.png" width="60" /> </td>
    <td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td>
    </tr>
    </table>

<!--
#### For more information about using NodePort see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/service-access-application-cluster/)
#### For more information about using LoadBalancer see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/load-balance-access-application-cluster/)
-->

<table style="font-family:Arial, Verdana, sans-serif; font-size: 20px;">
<tr>
   <td bgcolor="#ffffff" color="#ffffff" style="float:left "><img src="images/traefik.logo.png" width="100" /></td>
    <td bgcolor="#ffffff" color="#ffffff" style='float:middle;font-family:"Courier New", Courier, monospace; font-size:100%;' ><b>Traefik</b> see [https://traefik.io](https://traefik.io/)</td>
    </tr><tr>
    <td bgcolor="#ffffff" color="#ffffff" style='float:middle;font-family:"Courier New", Courier, monospace; font-size:100%;' ><img src="images/containous.logo.png" width="120" style="float:left" /></td>
    <td bgcolor="#ffffff" color="#ffffff" style="float:left" ><b>Containous</b>, the creators of Traefik, see [https://containo.us](https://containo.us)</td>
  </tr>
</table>

[INDEX](Kubernetes.ipynb)  [NEXT](4.5.Traefik.ipynb)

<img src="" width="80" style="float:left" />
<br/>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>